In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
df=pd.read_excel('chatbot.xlsx')
df.head()

,Context,Text Response
0,Tell me about your personality,Just think of me as the ace up your sleeve.
1,I want to know you better,I can help you work smarter instead of harder
2,Define yourself,Just think of me as the ace up your sleeve.
3,Describe yourself,I can help you work smarter instead of harder
4,tell me about yourself,Just think of me as the ace up your sleeve.


In [3]:
df.isnull().sum()

Context          0
Text Response    0
dtype: int64

In [4]:
import re
import nltk
from nltk import bigrams,trigrams,ngrams
from nltk.stem import wordnet
lema=wordnet.WordNetLemmatizer()
from nltk import pos_tag
from sklearn.metrics import pairwise_distances


# Text Normalization

In [5]:
def text_normalize(text):
    text=str(text).lower()
    text=re.sub(r'[^a-z0-9 ]','',text)
    tag_list=pos_tag(nltk.word_tokenize(text),tagset=None)
    lema_sent=[]
    for token,pos_token in tag_list:
                
        if pos_token.startswith("V"):#verb
            pos_val="v"
        elif pos_token.startswith("J"):#adjective
            pos_val='a'
        elif pos_token.startswith("R"):#adverb
            pos_val='r'
        else:#any parts of speech except verb,adjective,adverb
            pos_val='n'
        lema_token=lema.lemmatize(token,pos_val)#computing the parts of speech
        lema_sent.append(lema_token)
        
    return " ".join(lema_sent)

In [6]:
df['Context_clen']=df['Context'].apply(text_normalize)
df.head()

,Context,Text Response,Context_clen
0,Tell me about your personality,Just think of me as the ace up your sleeve.,tell me about your personality
1,I want to know you better,I can help you work smarter instead of harder,i want to know you good
2,Define yourself,Just think of me as the ace up your sleeve.,define yourself
3,Describe yourself,I can help you work smarter instead of harder,describe yourself
4,tell me about yourself,Just think of me as the ace up your sleeve.,tell me about yourself


# Word embedding

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [8]:
tfidf=TfidfVectorizer()

In [9]:
x_tfidf=tfidf.fit_transform(df['Context_clen']).toarray()
x_tfidf

array([[0.        , 0.        , 0.41438843, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.64126079,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [10]:
while(True):
    Q=input('Enter the Question :')
    if Q=='End':
        print("Bye,thank You,I Hope have a great conversation")
        break
    Q_clean=text_normalize(Q)
    Q_def=tfidf.transform([Q_clean]).toarray()
    cos=1-pairwise_distances(x_tfidf,Q_def,metric='cosine')
    ind=cos.argmax()
    if ind>0.2:
        result=df.loc[df['Context']==df['Context'].loc[ind],'Text Response']
        print('Text Response :',result.iloc[0])
    else:
        print("enter the correct question:")




Enter the Question :hi\
Text Response : Hey!
Enter the Question :rajesh
enter the correct question:
Enter the Question :Describe yourself
Text Response : I can help you work smarter instead of harder
Enter the Question :End
Bye,thank You,I Hope have a great conversation


In [ ]:
import pickle

In [ ]:
text_normal=pickle.dumps(text_normalize)